<a href="https://colab.research.google.com/github/Nillrze/QtCipherSqlitePlugin/blob/master/GITESTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import re
import string
import scipy
import pickle
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
#from prettytable import PrettyTable
#from joblib import dump, load
from astropy.table import Table, Column
import os
import glob
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from sklearn.feature_selection import VarianceThreshold


In [41]:
male_comments=" "
file_count = 0
for files in glob.glob('/content/drive/My Drive/dataEDGAD/Male/*.txt'):
   if file_count < 10:    # balanced
    file1 = open(files,encoding="utf8")
    male_comments += "Id : " + os.path.splitext(os.path.basename(files))[0] + "\n"
    male_comments += file1.read()
    file_count += 1


female_comments=" "
file_count = 0
for files in glob.glob('/content/drive/My Drive/dataEDGAD/Female/*.txt'):
  if file_count < 10:     # balanced
    file1 = open(files,encoding="utf8") 
    female_comments += "Id : " + os.path.splitext(os.path.basename(files))[0] + "\n" 
    female_comments += file1.read()
    file_count += 1

combine_data=male_comments+female_comments



In [42]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
def clean_tweets(tweet):
     #Remove URLs
    tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'https?://.*?\\s+', '', tweet, flags=re.MULTILINE)
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweet=pattern.sub('', tweet)
    #re.sub(r'(\s)http\w+', r'\1',tweet )
    #Remove long words
    longword = re.compile(r'\W*\b\w{15,500}\b')
    tweet=longword.sub('', tweet)
    #Remove short words
    shortword = re.compile(r'\W*\b\w{1,3}\b')
    tweet=shortword.sub('', tweet)
#remove emojis from tweet
    tweet = deEmojify(tweet)

    # lower capitalization
    tweet=tweet.lower()
    
    # Remove punctuation 
    remove = string.punctuation
    remove = remove.replace("@", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    tweet=re.sub(pattern, "", tweet) 
    
    #Remove username
    tweet=re.sub(r'@.*?(?=\s)', '', tweet)
    #Remove digits
    tweet = ''.join([i for i in tweet if not i.isdigit()])

    # tweet = tweet.split("\n")
    # for line in tweet:
    #     if line.startswith("tweet"):
    #         tweet = ''.join(line)

    # tweet = ''.join([i for i in tweet if i.startswith("tweet")])
            
  
    return tweet
# male_comments=clean_tweets(male_comments)
# female_comments=clean_tweets(female_comments)

In [43]:
#Remove URLs
#female_comments = re.sub(r"http\S+", "", female_comments)
#male_comments = re.sub(r"http\S+", "", male_comments)
#split
male_comments=male_comments.split("\n")
female_comments=female_comments.split("\n")

male_comments_tweet = ""
female_comments_tweet = ""
male_id = ""
female_id = ""
male_ids = ""
female_ids = ""
tweet_count = 0
# remove data and lang from text
for line in male_comments :
  if line.startswith("Id : "):
    #Get id:
    line = re.sub("Id : ", "",line)
    male_id = line
    tweet_count = 0
  if line.startswith("Tweet: ") and tweet_count < 80: # each user number of tweets
    #Remove Tweet:
    line = re.sub("Tweet: ", "",line)
    male_comments_tweet += line + "\n"
    male_ids += male_id + "\n"
    tweet_count += 1
  

for line in female_comments:
  if line.startswith("Id : "):
    #Get id:
    line = re.sub("Id : ", "",line)
    female_id = line 
    tweet_count = 0
  if line.startswith("Tweet: ")and tweet_count < 80: # each user number of tweets
    #Remove Tweet: 
    line = re.sub("Tweet: ", "",line)
    female_comments_tweet += line + "\n"
    female_ids += female_id + "\n"
    tweet_count += 1
  

male_comments=male_comments_tweet.split("\n")
female_comments=female_comments_tweet.split("\n")
male_ids_user=male_ids.split("\n")
female_ids_user=female_ids.split("\n")




In [25]:
#TODO: YOUR CODE GOES HERE
#comments=comments.replace('@', '[AT]', 1).replace('.', '').replace('[AT]', '#')

male_comments=pd.DataFrame(male_comments)
female_comments=pd.DataFrame(female_comments)
male_comments['gender']='male'
male_comments['idUser']=male_ids_user
female_comments['gender']='female'
female_comments['idUser']=female_ids_user
# male_comments = male_comments.sample(frac=1)
print(male_comments.head(10))
print(female_comments.head(10))

# male_comments_data=pd.DataFrame(male_comments_data)
# male_comments_data['gender']='male'
# # print(male_comments_datas)
# print(male_comments_data.head(10))

                                                   0 gender idUser
0  RT @MoSalah: كل عام وأنتم بخير ورمضان كريم ربن...   male       
1  RT @nourine_hammad: This drawing is a tribute ...   male      8
2                         ✋🤚 https://t.co/Rj2W1Z4gQE   male      8
3                          😂 https://t.co/ggkbGY5jAT   male      8
4  RT @MoSalah: It was a very tough night, but I'...   male      8
5                                               1h45   male      8
6                                               24h.   male      8
7                                       RT @LFC: 👏👏👏   male      8
8            RT @MoSalah: 🧐🧐 https://t.co/REESq5bg2t   male      8
9  RT @DanielAgger: Thanks for the visit boys😎hop...   male      8
                                                   0  gender idUser
0                           ازاي ؟؟ بجد ... ازاي ؟!!  female     95
1           @ahmedkadry92 @Ahmad_elAtriby بس يا اخ 😒  female     95
2                                   @Ahmad_elAtriby   femal

In [26]:
#Combine Male and female tweets
tweets = [male_comments, female_comments]
tweets = pd.concat(tweets)
tweets.index = range(len(tweets))
print(len(tweets))
tweets.rename(columns={0:'tweets'}, inplace=True)
print(tweets.head(5)['tweets'])

1413
0    RT @MoSalah: كل عام وأنتم بخير ورمضان كريم ربن...
1    RT @nourine_hammad: This drawing is a tribute ...
2                           ✋🤚 https://t.co/Rj2W1Z4gQE
3                            😂 https://t.co/ggkbGY5jAT
4    RT @MoSalah: It was a very tough night, but I'...
Name: tweets, dtype: object
1413
0    RT @MoSalah: كل عام وأنتم بخير ورمضان كريم ربن...
1    RT @nourine_hammad: This drawing is a tribute ...
2                           ✋🤚 https://t.co/Rj2W1Z4gQE
3                            😂 https://t.co/ggkbGY5jAT
4    RT @MoSalah: It was a very tough night, but I'...
Name: tweets, dtype: object


In [27]:
#Trian Test Split
x_train, x_test, y_train, y_test = train_test_split(tweets['tweets'], tweets['gender'], test_size=0.33, random_state=7)

In [28]:
print("""train Dataset:
=================================================================""")
train_dataset=pd.DataFrame(x_train)
train_dataset['gender']=pd.DataFrame(y_train)
print(train_dataset.head(10))

train Dataset:
                                                tweets  gender
130  "لا حيلة لك في حزن امرأة تدس كل أوجاعها في ضحكة."    male
168  RT @Alkhouzami: من فترة كان في أزمة تخص اتوبيس...    male
264  RT @jrbiboo: ويطلقك بعد بصتك لعمرو https://t.c...    male
586  زا عبدول شو المقدمة الحلقة اللي مش الأولي The ...    male
501                       إنجلترا هربت نيك بالتعادل ده    male
955  RT @mosalehu: سور اي بلكونة او سطوح قصير https...  female
272  @ihejazii @hossamfaid مش كنت دخلت حربية كنت دل...    male
922                  RT @EmrgencyKittens: Jail break!   female
420                        @hegshj34 دا رقمهم 22618691    male
94   "وتبقى لحظاتُ الصباح الأولى هي الأصعبُ دائمًا،...    male
train Dataset:
                                                tweets  gender
130  "لا حيلة لك في حزن امرأة تدس كل أوجاعها في ضحكة."    male
168  RT @Alkhouzami: من فترة كان في أزمة تخص اتوبيس...    male
264  RT @jrbiboo: ويطلقك بعد بصتك لعمرو https://t.c...    male
586  زا عبدول شو المقدمة 

In [29]:
print("""test Dataset:
================================================================""")
test_dataset=pd.DataFrame(x_test)
test_dataset['gender']=pd.DataFrame(y_test)
print(test_dataset.head(10))

test Dataset:
                                                 tweets  gender
525                   كسم قلة أدب و حلاوة كاس العالم ده    male
1082                   @Hopss_ alah yebarek feekkk😘❤️❤️  female
715   RT @bunnyarchive: got a new paper shredder htt...  female
1356                       @BEBO1911 Aw le7ad ma te7boh  female
1277  RT @AliaaAsallam: هو ليه في لون كحلي ومفيش لون...  female
536                                  مش هاسرّح شعري !!!    male
454   I need a cure for a serious condition I got. S...    male
1026                  @RamiElhashimy @iMuhndawi meen di  female
432                            @aladdin_mostafa تفتكر ؟    male
325      RT @Tegovic: @7omous1  https://t.co/puv3ApzXUy    male
test Dataset:
                                                 tweets  gender
525                   كسم قلة أدب و حلاوة كاس العالم ده    male
1082                   @Hopss_ alah yebarek feekkk😘❤️❤️  female
715   RT @bunnyarchive: got a new paper shredder htt...  female
1356        

In [30]:
print(""" 'Male' in train data:
============================================================""")
male_train_dataset=train_dataset[train_dataset.gender=='male']
print(male_train_dataset.head(10))

print("""Words used by 'feMale' in train data:
============================================================""")
female_train_dataset=train_dataset[train_dataset.gender=='female']
print(female_train_dataset.head(10))

 'Male' in train data:
                                                tweets gender
130  "لا حيلة لك في حزن امرأة تدس كل أوجاعها في ضحكة."   male
168  RT @Alkhouzami: من فترة كان في أزمة تخص اتوبيس...   male
264  RT @jrbiboo: ويطلقك بعد بصتك لعمرو https://t.c...   male
586  زا عبدول شو المقدمة الحلقة اللي مش الأولي The ...   male
501                       إنجلترا هربت نيك بالتعادل ده   male
272  @ihejazii @hossamfaid مش كنت دخلت حربية كنت دل...   male
420                        @hegshj34 دا رقمهم 22618691   male
94   "وتبقى لحظاتُ الصباح الأولى هي الأصعبُ دائمًا،...   male
42   @JanAageFjortoft @LFC @ASRomaEN @ChampionsLeag...   male
56   "أرى أنهُ من العذوبة، والخدر، والاجتياح، أن نص...   male
Words used by 'feMale' in train data:
                                                 tweets  gender
955   RT @mosalehu: سور اي بلكونة او سطوح قصير https...  female
922                   RT @EmrgencyKittens: Jail break!   female
961   مش عارفة أجيبها للبنات الكيوت إزاي إن إسمها "م...  female
9

In [31]:
dict_words_male=defaultdict(int)
count=0
for x in male_train_dataset['tweets']:
    words=x.split(" ")
    for y in words:
        dict_words_male[y]=dict_words_male[y]+1

print(" MALE. :  " , dict_words_male)

dict_words_female=defaultdict(int)
count=0
for x in female_train_dataset['tweets']:
    words=x.split(" ")
    for y in words:
        dict_words_female[y]=dict_words_female[y]+1

print(" FEMALE. :  " , dict_words_female)

 MALE. :   defaultdict(<class 'int'>, {'"لا': 2, 'حيلة': 1, 'لك': 5, 'في': 48, 'حزن': 1, 'امرأة': 1, 'تدس': 1, 'كل': 19, 'أوجاعها': 1, 'ضحكة."': 1, 'RT': 73, '@Alkhouzami:': 1, 'من': 44, 'فترة': 1, 'كان': 7, 'أزمة': 1, 'تخص': 1, 'اتوبيس': 1, 'منتخب': 1, 'مصر': 5, 'اللي': 18, 'شكله': 2, 'زي': 1, 'الزفت': 1, '،': 6, 'شاب': 1, 'محترم': 1, 'عمل': 1, 'تصميم': 1, 'للاتوبيس': 1, 'بدون': 3, 'اي': 4, 'مقابل': 1, 'ومصر': 1, 'كلها': 2, 'ش…': 1, '@jrbiboo:': 1, 'ويطلقك': 1, 'بعد': 9, 'بصتك': 1, 'لعمرو': 1, 'https://t.co/Pwklpt1VHp': 1, 'زا': 1, 'عبدول': 1, 'شو': 1, 'المقدمة': 1, 'الحلقة': 1, 'مش': 27, 'الأولي': 1, 'The': 5, 'Abdoul': 1, 'Show': 1, 'Intro:': 1, 'https://t.co/NDlfbwuBXK': 1, 'via': 1, 'إنجلترا': 1, 'هربت': 1, 'نيك': 1, 'بالتعادل': 1, 'ده': 11, '@ihejazii': 5, '@hossamfaid': 6, 'كنت': 11, 'دخلت': 3, 'حربية': 1, 'دلعتنى': 1, ':(': 1, '@hegshj34': 3, 'دا': 10, 'رقمهم': 1, '22618691': 1, '"وتبقى': 1, 'لحظاتُ': 1, 'الصباح': 1, 'الأولى': 2, 'هي': 4, 'الأصعبُ': 1, 'دائمًا،': 1, 'حين': 4, '

In [32]:
# fit method
def text_fit(X, y, model,ml_model):
    
    X_c = model.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=0)
    ml = ml_model.fit(X_train, y_train)
    
    y_pred = ml_model.predict(X_test)
    print("accuracy score of ",accuracy_score(y_test, y_pred))
    # auc=roc_auc_score(y_test, ml_model.predict_proba(X_test)[:,1])
    # print("AUC score",auc)
    # print('')
    # print(classification_report(y_test, y_pred, target_names=["negative", "positive"]))

In [33]:

#RandomForest model on TFIDF Uni-gram
tfidf = TfidfVectorizer()
text_fit(tweets['tweets'],tweets['gender'], tfidf, RandomForestClassifier())


accuracy score of  0.7259887005649718
accuracy score of  0.7259887005649718


In [34]:
#RandomForest model on TFIDF + ngram 1-2
tfidf_n = TfidfVectorizer(ngram_range=(1,2))
text_fit(tweets['tweets'],tweets['gender'], tfidf_n, RandomForestClassifier())

accuracy score of  0.6864406779661016
accuracy score of  0.7175141242937854


In [35]:
#RandomForest model on TFIDF + ngram 1-3
tfidf_nc = TfidfVectorizer(ngram_range=(1,3))
text_fit(tweets['tweets'],tweets['gender'], tfidf_nc, RandomForestClassifier())

accuracy score of  0.6892655367231638
accuracy score of  0.6779661016949152


In [36]:
# RandomForest with Uni-gram BoW and stop_words
cv = CountVectorizer()   
text_fit(tweets['tweets'],tweets['gender'], cv, RandomForestClassifier())

accuracy score of  0.7259887005649718
accuracy score of  0.711864406779661


In [37]:
# RandomForest on CV + ngram 1-2
cv_n = CountVectorizer(ngram_range=(1,2))   
text_fit(tweets['tweets'],tweets['gender'], cv_n, RandomForestClassifier())

accuracy score of  0.711864406779661
accuracy score of  0.7033898305084746


In [38]:
# RandomForest on CV + ngram 1-3
cv_nc = CountVectorizer(ngram_range=(1,3))   
text_fit(tweets['tweets'],tweets['gender'], cv_nc, RandomForestClassifier())

accuracy score of  0.7146892655367232
accuracy score of  0.7146892655367232


In [39]:

# Logistic regression model on TFIDF
tfidf = TfidfVectorizer()
text_fit(tweets['tweets'],tweets['gender'], tfidf, LogisticRegression(penalty='l2'))

accuracy score of  0.7372881355932204
accuracy score of  0.7372881355932204


In [40]:
print("""Train Dataset Labels Encoding:

=============================================""")

le = LabelEncoder()
le.fit(train_dataset["gender"])
train_dataset['encoded_gender']=le.transform(train_dataset["gender"])
print(train_dataset.head(10))

print("""Test Dataset Labels Encoding:

=============================================""")

le.fit(test_dataset["gender"])
test_dataset['encoded_gender']=le.transform(test_dataset["gender"])
print(test_dataset.head(10))

Train Dataset Labels Encoding:

                                                tweets  gender  encoded_gender
130  "لا حيلة لك في حزن امرأة تدس كل أوجاعها في ضحكة."    male               1
168  RT @Alkhouzami: من فترة كان في أزمة تخص اتوبيس...    male               1
264  RT @jrbiboo: ويطلقك بعد بصتك لعمرو https://t.c...    male               1
586  زا عبدول شو المقدمة الحلقة اللي مش الأولي The ...    male               1
501                       إنجلترا هربت نيك بالتعادل ده    male               1
955  RT @mosalehu: سور اي بلكونة او سطوح قصير https...  female               0
272  @ihejazii @hossamfaid مش كنت دخلت حربية كنت دل...    male               1
922                  RT @EmrgencyKittens: Jail break!   female               0
420                        @hegshj34 دا رقمهم 22618691    male               1
94   "وتبقى لحظاتُ الصباح الأولى هي الأصعبُ دائمًا،...    male               1
Test Dataset Labels Encoding:

                                                 tweets  gender  enc

In [ ]:
print("""Bag_of_words:
============================================================""")
Bag_of_words=defaultdict(int)
count=0
f = open("Bag of words.csv","w")
for x in train_dataset['tweets']:
    words=x.split(" ")
    for y in words:
        Bag_of_words[y]=Bag_of_words[y]+1
for key, value in Bag_of_words.items():
    data=key+','
    f.write(data)
f.close()